<a href="https://colab.research.google.com/github/Rodrigoveloso7/CONDUCAO1D/blob/main/Dimensionamento_Desumidificador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalação bibliotecas

In [323]:
# pip install CoolProp


In [324]:
# pip install ht

In [325]:
import numpy as np
from CoolProp.CoolProp import PropsSI
import ht as ht
import ht.boiling_flow as bf

#Condições de operação Biogás

In [326]:
fracoes_volumetricas = {"Methane":0.60,"CarbonDioxide":0.35,"Water":0.05} # Frações volumétricas dos componentes do biogás
# fracoes_volumetricas = {"Nitrogen":0.7691,"Oxygen":0.2078,'Argon':0.0092,'CarbonDioxide':0.0004,'Water':0.017} # Frações volumétricas dos componentes do biogás
# fracoes_volumetricas = {"Methane":0.637,"CarbonDioxide":0.35,"Water":0.013}

In [327]:
P_entrada_biogas = 33.34*1000 # [Pa] - Pressão de admissão do biogás
T_entrada_biogas = 273.15+ 73.4# [K] - Temperatura de admissão do biogás
T_referencia = 273.15 # [K] - Temperatura de referência
P_referencia = 101325 # [Pa] - Pressão de referência
vazao_normal_biogas = 60 # [Nm³/h] - Vazão volumétrica normalizada do biogás
T_alvo_biogas = 273.15 # [K] - Temperatura pretendida do biogás na saída
T_alvo_arrefecimento = 273.15-1 # [K] - Temperatura pretendida do líquido de arrefecimento
T_ambiente = 273.15+25 # [K] - Temperatura Ambiente
P_ambiente = 101325 # [Pa] - Pressão ambiente

#Propriedades geométricas casco

In [328]:
diam_casco = 330/1000 # m Diâmetro interno do casco
diam_entrada_arrefecimento = 19.05/1000 # m Diâmetro de entrada do fluido de arrefecimento
comprimento_casco = 1995/1000 # m Comprimento do casco
volume_arrefecimento = ((np.pi*((diam_casco)**2))/4)*comprimento_casco # m³ Volume do fluido de arrefecimento (mistura água + monoetilenoglicol)




# Propriedades geométricas serpentina

In [329]:
diam_hel_serp = 270/1000 # m Diâmetro da espira
dext_serp = 12.7/1000 # m Diâmetro externo da serpentina
espessura_serp = 1.2/1000 # m Espessura da serpentina
diam_interno_serp = dext_serp-2*espessura_serp # m Diâmetro interno da serpentina

#Condições de operação líquido de arrefecimento

https://www.engineeringtoolbox.com/ethylene-glycol-d_146.html

Considerando uma mistura 80% água e 20% monoetilenoglicol para obter uma temperatura de solidificação de -7.9C

In [330]:
cp_arrefecimento = 3.951*1000 # J/kg.K
densidade_arrefecimento = 1030 # kg/m³
viscosidade_dinamica_arrefecimento = 1.42/1000 # Pa.s Este valor foi obtido em cima de fracao massica
k_arrefecimento = 0.55 # W/mK
massa_arrefecimento = volume_arrefecimento*densidade_arrefecimento #kg
massa_arrefecimento
tempo_arrefecimento = 60*40 # segundos

In [331]:
vazao_volumetrica_bomba = 1.8/3600 #m³/s
vazao_massica_bomba = vazao_volumetrica_bomba*densidade_arrefecimento
u_casco = vazao_volumetrica_bomba/((np.pi*((diam_entrada_arrefecimento))**(2))/4)
u_casco = u_casco*0.5

In [332]:
vazao_volumetrica_biogas_entrada = (vazao_normal_biogas*(P_referencia/P_entrada_biogas)*(T_entrada_biogas/T_referencia))/3600 #m³/s
massa_molar_total_entrada = 0
for elementos in fracoes_volumetricas:
    massa_molar_total_entrada = massa_molar_total_entrada + fracoes_volumetricas[elementos]*PropsSI('M', 'T', T_entrada_biogas, 'P', P_entrada_biogas, elementos)
densidade_operacao_biogas = (P_entrada_biogas*massa_molar_total_entrada)/(8.314*T_entrada_biogas)
vazao_massica_biogas = vazao_volumetrica_biogas_entrada*densidade_operacao_biogas


In [333]:
densidade_operacao_biogas

0.3000475386512928

In [334]:
massa_molar_total = 0
cp_biogas = 0
for elementos in fracoes_volumetricas:
  massa_molar_total = massa_molar_total + fracoes_volumetricas[elementos]*PropsSI('M', 'T', T_entrada_biogas, 'P', P_entrada_biogas, elementos)
massa_molar_total

0.0259298734

#Função para definição de calor específico do biogás em função da pressão e temperatura

In [335]:
def cp_biogas(P_biogas,T_biogas):
   massa_molar_total = 0
   cp_biogas = 0
   for elementos in fracoes_volumetricas:
    massa_molar_total = massa_molar_total + fracoes_volumetricas[elementos]*PropsSI('M', 'T', T_biogas, 'P', P_biogas, elementos)
   #print(massa_molar_total)
   for elementos in fracoes_volumetricas:
    cp_biogas = cp_biogas + (((PropsSI('M', 'T', T_biogas, 'P', P_biogas*fracoes_volumetricas[elementos], elementos)*fracoes_volumetricas[elementos])/massa_molar_total)*PropsSI('C', 'T', T_biogas, 'P', P_biogas*fracoes_volumetricas[elementos], elementos))
   return cp_biogas

#Função para determinação de carga termica

In [336]:
def carga_termica():
  divisoes = 200
  Qsens_biogas = 0
  T = np.linspace(T_entrada_biogas,T_alvo_biogas,divisoes)
  for i in range(len(T)-1):
    cp = cp_biogas(P_entrada_biogas,(T[i]+T[i+1])/2)
    Qsens_biogas = vazao_massica_biogas*cp*(T[i] - T[i+1]) + Qsens_biogas
    cp= 0
  Qlat_biogas = vazao_massica_biogas*(fracoes_volumetricas['Water']*PropsSI('M', 'T', T_entrada_biogas, 'P', P_entrada_biogas, elementos)/massa_molar_total_entrada)*(((PropsSI("H", "T", T_alvo_biogas, "Q", 1, "Water")) - PropsSI("H", "T", T_alvo_biogas, "Q", 0, "Water")))
  Q_arrefecimento = massa_arrefecimento*cp_arrefecimento*(T_ambiente - T_alvo_arrefecimento)/tempo_arrefecimento
  Qtotal = Qlat_biogas + Qsens_biogas +Q_arrefecimento
  return Qtotal,Qlat_biogas,Qsens_biogas,Q_arrefecimento

In [337]:
carga_termica()

(np.float64(11219.792543233192),
 1675.2046467199623,
 np.float64(2022.004314721665),
 7522.583581791565)

#Calculando comprimento evaporador

Definição da vazão mássica a partir da carga térmica, utilizando software CoolSelector da Danfoss


In [338]:
vazao_massica_refri = 240.3/3600 #kg/s
T_evap = 273.15-10 # K
P_entrada_evap = 235.4*1000 #Pa
x_ini = 0.3 # fração de vapor na entrada do evaporador
Q_lat_refr = vazao_massica_refri*(((PropsSI("H", "T", T_evap, "Q", 1, "R22")) - PropsSI("H", "T", T_evap, "Q", 0.3, "R22")))
Q_lat_refr # W

9942.75827141813

In [339]:
T_sat = T_evap
fluid = 'R22'
divisoes = 100
superaquecimento = 20
T_range_superaquecimento = np.linspace(T_sat,T_sat+ superaquecimento,divisoes)
k_inox = 16
Re_casco = (densidade_arrefecimento*u_casco*diam_hel_serp)/(viscosidade_dinamica_arrefecimento)
Pr_casco = (cp_arrefecimento*viscosidade_dinamica_arrefecimento)/(k_arrefecimento)

https://www.sciencedirect.com/science/article/abs/pii/S0140700714002369

https://jingweizhu.weebly.com/uploads/1/3/5/4/13548262/summary_of_chen%E2%80%99s_correlation_for_boiling_heat_transfer_to_saturated_fluids_in_convective_flow.pdf

Esse e descrição do chat gpt


In [340]:
# Propriedades obtidas na linha de saturação
rhol = PropsSI('D', 'T', T_evap, 'Q', 0, fluid)       # kg/m³
rhog = PropsSI('D', 'T', T_evap, 'Q', 1, fluid)       # kg/m³
mul  = PropsSI('V', 'T', T_evap, 'Q', 0, fluid)       # Pa.s
mug  = PropsSI('V', 'T', T_evap, 'Q', 1, fluid)       # Pa.s
kl   = PropsSI('L', 'T', T_evap, 'Q', 0, fluid)       # W/m.K
Cpl  = PropsSI('C', 'T', T_evap, 'Q', 0, fluid)       # J/kg.K
Hvap = PropsSI('H', 'T', T_evap, 'Q', 1, fluid) - \
      PropsSI('H', 'T', T_evap, 'Q', 0, fluid)       # J/kg
sigma = PropsSI('SURFACE_TENSION', 'T', T_evap, 'Q', 0, fluid)  # N/m


x = np.linspace(x_ini,0.99, divisoes)
x

array([0.3       , 0.3069697 , 0.31393939, 0.32090909, 0.32787879,
       0.33484848, 0.34181818, 0.34878788, 0.35575758, 0.36272727,
       0.36969697, 0.37666667, 0.38363636, 0.39060606, 0.39757576,
       0.40454545, 0.41151515, 0.41848485, 0.42545455, 0.43242424,
       0.43939394, 0.44636364, 0.45333333, 0.46030303, 0.46727273,
       0.47424242, 0.48121212, 0.48818182, 0.49515152, 0.50212121,
       0.50909091, 0.51606061, 0.5230303 , 0.53      , 0.5369697 ,
       0.54393939, 0.55090909, 0.55787879, 0.56484848, 0.57181818,
       0.57878788, 0.58575758, 0.59272727, 0.59969697, 0.60666667,
       0.61363636, 0.62060606, 0.62757576, 0.63454545, 0.64151515,
       0.64848485, 0.65545455, 0.66242424, 0.66939394, 0.67636364,
       0.68333333, 0.69030303, 0.69727273, 0.70424242, 0.71121212,
       0.71818182, 0.72515152, 0.73212121, 0.73909091, 0.74606061,
       0.7530303 , 0.76      , 0.7669697 , 0.77393939, 0.78090909,
       0.78787879, 0.79484848, 0.80181818, 0.80878788, 0.81575

CORRELAÇÃO CHEN

In [341]:
def correlacao_chen(x,fluid):
  # Fluido e condições de operação
  fluid = fluid
  T_sat = T_evap  # Temperatura de saturação em K (100 °C)
  Te = 8        # Superaquecimento da parede (T_wall - T_sat) em K         # Título de vapor
  # Derivada da pressão de saturação em função da temperatura (dPsat/dT)
  delta_T = 0.01  # K
  p1 = PropsSI('P', 'T', T_sat - delta_T, 'Q', 0, fluid)
  p2 = PropsSI('P', 'T', T_sat + delta_T, 'Q', 0, fluid)
  dPsat = (p2 - p1) / (2 * delta_T)  # Pa/K

  # Cálculo do coeficiente de convecção
  h = bf.Chen_Bennett(vazao_massica_refri, x, diam_interno_serp, rhol, rhog, mul, mug, kl, Cpl, Hvap, sigma, dPsat, Te)
  return h


CORRELAÇÃO CHURCHILL BERNSTEIN

In [342]:
def correlacao_churchill_bernstein(Re, Pr):
  nu_ext = ht.conv_external.Nu_cylinder_Churchill_Bernstein(Re, Pr)
  h_externo = nu_ext*k_arrefecimento/(diam_hel_serp)
  return h_externo

In [343]:
def correlacao_dittus_boelter(Re, Pr,k):

  Nu = ht.conv_internal.turbulent_Dittus_Boelter(Re, Pr, heating=False, revised=True)
  h = Nu*k/diam_interno_serp
  return h


In [344]:
L_evap_vapor = 0
for i in range(len(T_range_superaquecimento)-1):
  Q_sup = vazao_massica_refri*PropsSI('C', 'T', (T_range_superaquecimento[i]+T_range_superaquecimento[i+1])/2, 'P', P_entrada_evap, 'R22')*(T_range_superaquecimento[i+1]-T_range_superaquecimento[i])
  velocidade_refrigerante_vapor = vazao_massica_refri/((np.pi*(diam_interno_serp**2)/4)*(PropsSI('D', 'T', T_range_superaquecimento[i]+T_range_superaquecimento[i+1]/2, 'P', P_entrada_evap, 'R22')))
  reynolds_vapor_refr = (velocidade_refrigerante_vapor*diam_interno_serp*PropsSI('D', 'T', (T_range_superaquecimento[i]+T_range_superaquecimento[i+1])/2, 'P', P_entrada_evap, 'R22'))/(PropsSI('V', 'T', T_range_superaquecimento[i]+T_range_superaquecimento[i+1]/2, 'P', P_entrada_evap, 'R22'))
  Pr_vapor = (PropsSI('C', 'T', (T_range_superaquecimento[i]+T_range_superaquecimento[i+1])/2, 'P', P_entrada_evap, 'R22')*PropsSI('V', 'T', (T_range_superaquecimento[i]+T_range_superaquecimento[i+1])/2, 'P', P_entrada_evap, 'R22'))/ (PropsSI('L', 'T', (T_range_superaquecimento[i]+T_range_superaquecimento[i+1])/2, 'P', P_entrada_evap, 'R22'))
  L_evap_vapor_temp = Q_sup*(((np.log(dext_serp/diam_interno_serp))/(2*np.pi*k_inox))+(1/correlacao_dittus_boelter(reynolds_vapor_refr, Pr_vapor,PropsSI('L', 'T', (T_range_superaquecimento[i]+T_range_superaquecimento[i+1])/2, 'P', P_entrada_evap, 'R22'))*np.pi*diam_interno_serp))+(1/(correlacao_churchill_bernstein(Re_casco, Pr_casco)*np.pi*dext_serp))*(1/(T_alvo_arrefecimento-T_evap))
  L_evap_vapor = L_evap_vapor + L_evap_vapor_temp
L_evap_vapor

np.float64(2.0405577102888537)

In [345]:
L_evap = 0
for i in range(len(x)-1):
  Q_lat_refr = vazao_massica_refri*(((PropsSI("H", "T", T_evap, "Q", x[i+1], "R22")) - PropsSI("H", "T", T_evap, "Q", x[i], "R22")))
  L_evap_temp = Q_lat_refr*(((np.log(dext_serp/diam_interno_serp))/(2*np.pi*k_inox))+(1/(correlacao_chen(x[i],'R22')*np.pi*diam_interno_serp))+(1/(correlacao_churchill_bernstein(Re_casco, Pr_casco)*np.pi*dext_serp)))*(1/(T_alvo_arrefecimento-T_evap))
  L_evap = L_evap + L_evap_temp
L_evap

np.float64(21.06261721428279)

In [346]:
L_serpentina = L_evap+L_evap_vapor
L_serpentina

np.float64(23.103174924571643)

Cálculo região de superaquecimento

#Dimensionamento tubos colméia

#Propriedades geométricas colméia

In [347]:
dext_tubos = 19.05/1000 # m
espessura_tubos = 1.2/1000 # m
dint_tubos = dext_tubos-2*espessura_tubos # m
A_int_tubos = np.pi*(dint_tubos**2)/4 # m²
A_ext_tubos = np.pi*(dext_tubos**2)/4 # m²
N = 66

#Função para definição da viscosidade dinâmica do biogás em função da temperatura e pressão

In [348]:
def visc_dinamica_biogas(P_biogas,T_biogas):
   massa_molar_total = 0
   visc_dinamica_biogas = 0
   for elementos in fracoes_volumetricas:
    massa_molar_total = massa_molar_total + fracoes_volumetricas[elementos]*PropsSI('M', 'T', T_biogas, 'P', P_biogas, elementos)
   #print(massa_molar_total)
   for elementos in fracoes_volumetricas:
    visc_dinamica_biogas = visc_dinamica_biogas + (((PropsSI('M', 'T', T_biogas, 'P', P_biogas*fracoes_volumetricas[elementos], elementos)*fracoes_volumetricas[elementos])/massa_molar_total)*PropsSI('V', 'T', T_biogas, 'P', P_biogas*fracoes_volumetricas[elementos], elementos))
   return visc_dinamica_biogas

#Função para definição da condutividade térmica do biogás em função da temperatura e pressão

In [349]:
def condutividade_biogas(P_biogas,T_biogas):
   massa_molar_total = 0
   condutividade_biogas = 0
   for elementos in fracoes_volumetricas:
    massa_molar_total = massa_molar_total + fracoes_volumetricas[elementos]*PropsSI('M', 'T', T_biogas, 'P', P_biogas, elementos)
   #print(massa_molar_total)
   for elementos in fracoes_volumetricas:
    condutividade_biogas = condutividade_biogas + (((PropsSI('M', 'T', T_biogas, 'P', P_biogas*fracoes_volumetricas[elementos], elementos)*fracoes_volumetricas[elementos])/massa_molar_total)*PropsSI('L', 'T', T_biogas, 'P', P_biogas*fracoes_volumetricas[elementos], elementos))
   return condutividade_biogas

#Função para definição de densidade do biogás em função da temperatura e pressão

In [350]:
def densidade_biogas(P_biogas,T_biogas):
   massa_molar_total = 0
   densidade_biogas = 0
   for elementos in fracoes_volumetricas:
    massa_molar_total = massa_molar_total + fracoes_volumetricas[elementos]*PropsSI('M', 'T', T_biogas, 'P', P_biogas, elementos)
   #print(massa_molar_total)
   densidade_biogas = (P_biogas*massa_molar_total)/(8.314*T_biogas)
   return densidade_biogas

In [351]:
P_entrada_biogas
T_entrada_biogas

346.54999999999995

In [352]:
# Rt_cond_tubos = np.log(dext_tubos/dint_tubos)/(2*np.pi*k_inox)
# vazao_massica_tubos = (vazao_massica_biogas/N)
# print(vazao_massica_biogas)
# print(vazao_massica_tubos)
# Q=0
# L_tubos_sen = 0
# divisoes = 100
# T = np.linspace(T_entrada_biogas,T_alvo_biogas,divisoes)
# print(P_entrada_biogas)
# for i in range(len(T)-1):
#   vel_tubos = (vazao_massica_tubos/(((densidade_biogas(P_entrada_biogas,(T[i+1]+T[i])/2)))*A_int_tubos))
#   # print(vel_tubos)
#   reynolds_biogas_sens = ((vel_tubos*dint_tubos*densidade_biogas(P_entrada_biogas,(T[i+1]+T[i])/2)))/(visc_dinamica_biogas(P_entrada_biogas,(T[i+1]+T[i])/2))
#   # print(reynolds_biogas_sens)
#   Pr_biogas_sens = (cp_biogas(P_entrada_biogas,(T[i+1]+T[i])/2) *visc_dinamica_biogas(P_entrada_biogas,(T[i+1]+T[i])/2) )/(condutividade_biogas(P_entrada_biogas,(T[i+1]+T[i])/2))

#   nud_db_biogas_sens = 0.023*(reynolds_biogas_sens**(0.8))*(Pr_biogas_sens**(0.4))
#   nud_db_biogas_sens
#   h_conv_tubos_sens = (nud_db_biogas_sens*condutividade_biogas(P_entrada_biogas,(T[i+1]+T[i])/2)/(dint_tubos))*1.3
#   fator_atrito = (64/reynolds_biogas_sens)

#   # print(h_conv_tubos_sens)
#   # print(h_conv_tubos_sens)
#   Rt_conv_tubos = (1/(h_conv_tubos_sens*np.pi*dint_tubos))
#   Q_bio_sens = vazao_massica_tubos*cp_biogas(P_entrada_biogas,(T[i+1]+T[i])/2)*(-T[i+1]+T[i])
#   L = Q_bio_sens*(Rt_cond_tubos+Rt_conv_tubos)/((((T[i+1]+T[i])/2)-T_alvo_arrefecimento))
#   Q = Q_bio_sens + Q
#   L_tubos_sen = L_tubos_sen + L
#   queda_pressao = fator_atrito*L*densidade_biogas(P_entrada_biogas,(T[i+1]+T[i])/2)*(vel_tubos**2)/(2*dint_tubos)
#   P_entrada_biogas = P_entrada_biogas - queda_pressao
#   # print(queda_pressao)
#   # print(Q
#   L =0
#   # print(Q)
# print(L_tubos_sen)
# print(P_entrada_biogas)
# print(Q)

CÁLCULO COMPRIMENTO COLMÉIA

In [353]:
NT = 7
ST = 30/1000
SL = 30/1000

In [354]:
def correlacao_zukauskas_bejan(Re, Pr, tube_rows, pitch_parallel, pitch_normal):
  Nu = ht.conv_tube_bank.Nu_Zukauskas_Bejan(Re, Pr, tube_rows, pitch_parallel, pitch_normal, Pr_wall=None)
  h = Nu*k_arrefecimento/(dext_tubos)
  return h

COMPRIMENTO SEM CONSIDERAR CONDENSAÇÃO DA ÁGUA

In [355]:
Rt_cond_tubos = np.log(dext_tubos/dint_tubos)/(2*np.pi*k_inox)
vazao_massica_tubos = (vazao_massica_biogas/N)
print(vazao_massica_biogas)
print(vazao_massica_tubos)
Q=0
L_tubos_sen = 0
T = np.linspace(T_entrada_biogas,T_alvo_biogas,divisoes)
for i in range(len(T)-1):
  vel_tubos = (vazao_massica_tubos/(((densidade_biogas(P_entrada_biogas,(T[i+1]+T[i])/2)))*A_int_tubos))
  reynolds_biogas_sens = ((vel_tubos*dint_tubos*densidade_biogas(P_entrada_biogas,(T[i+1]+T[i])/2)))/(visc_dinamica_biogas(P_entrada_biogas,(T[i+1]+T[i])/2))
  Pr_biogas_sens = (cp_biogas(P_entrada_biogas,(T[i+1]+T[i])/2) *visc_dinamica_biogas(P_entrada_biogas,(T[i+1]+T[i])/2) )/(condutividade_biogas(P_entrada_biogas,(T[i+1]+T[i])/2))
  Rt_conv_int_tubos = 1/(correlacao_dittus_boelter(reynolds_biogas_sens,Pr_biogas_sens,condutividade_biogas(P_entrada_biogas,(T[i+1]+T[i])/2))*np.pi*dint_tubos)
  Rt_conv_ext_tubos = (1/(correlacao_zukauskas_bejan(Re_casco,Pr_casco,NT,ST,SL)*np.pi*dext_tubos))
  Q_bio_sens = vazao_massica_tubos*cp_biogas(P_entrada_biogas,(T[i+1]+T[i])/2)*(-T[i+1]+T[i])
  L = Q_bio_sens*(Rt_conv_int_tubos+Rt_cond_tubos+Rt_conv_ext_tubos)/((((T[i+1]+T[i])/2)-T_alvo_arrefecimento))
  Q = Q_bio_sens + Q
  L_tubos_sen = L_tubos_sen + L
L_tubos_sen

0.019282108992257983
0.0002921531665493634


np.float64(2.462675035701003)

COMPRIMENTO PARA CONDENSAR VAPOR D'ÁGUA

In [356]:
# Propriedades obtidas na linha de saturação
rhol_vp_agua = PropsSI('D', 'T', T_alvo_biogas, 'Q', 0, 'Water')       # kg/m³
rhog_vp_agua = PropsSI('D', 'T', T_alvo_biogas, 'Q', 1, 'Water')       # kg/m³
mul_vp_agua  = PropsSI('V', 'T', T_alvo_biogas, 'Q', 0, 'Water')       # Pa.s
mug_vp_agua  = PropsSI('V', 'T', T_alvo_biogas, 'Q', 1, 'Water')       # Pa.s
kl_vp_agua   = PropsSI('L', 'T', T_alvo_biogas, 'Q', 0, 'Water')       # W/m.K
Cpl_vp_agua  = PropsSI('C', 'T', T_alvo_biogas, 'Q', 0, 'Water')       # J/kg.K
Hvap_vp_agua = PropsSI('H', 'T', T_alvo_biogas, 'Q', 1, 'Water') - \
      PropsSI('H', 'T', T_alvo_biogas, 'Q', 0, 'Water')       # J/kg
sigma_vp_agua = PropsSI('SURFACE_TENSION', 'T', T_alvo_biogas, 'Q', 0, 'Water')  # N/m

In [357]:
def correlacao_chen_vapor_agua(x,fluid):
  # Fluido e condições de operação
  fluid = fluid
  T_sat = T_alvo_biogas  # Temperatura de saturação em K (100 °C)
  Te = 8        # Superaquecimento da parede (T_wall - T_sat) em K         # Título de vapor
  # Derivada da pressão de saturação em função da temperatura (dPsat/dT)
  delta_T = 0.01  # K
  p1 = PropsSI('P', 'T', T_sat - delta_T, 'Q', 0, fluid)
  p2 = PropsSI('P', 'T', T_sat + delta_T, 'Q', 0, fluid)
  dPsat = (p2 - p1) / (2 * delta_T)  # Pa/K

  # Cálculo do coeficiente de convecção
  h = bf.Chen_Bennett(vazao_massica_vapor_agua, x, dint_tubos, rhol_vp_agua, rhog_vp_agua, mul_vp_agua, mug_vp_agua, kl_vp_agua, Cpl_vp_agua, Hvap_vp_agua, sigma_vp_agua, dPsat, Te)
  return h

In [358]:
L_tubo_condensacao = 0
Q_cond= 0
x_vapor_agua = np.linspace(0.01,0.99,divisoes)
vazao_massica_vapor_agua = (vazao_massica_biogas*((PropsSI('M', 'T', T_alvo_biogas, 'P', P_entrada_biogas*fracoes_volumetricas['Water'], 'Water')*fracoes_volumetricas['Water'])/massa_molar_total))/N
for i in range(len(x)-1):
  Q_lat_vapor_agua = vazao_massica_vapor_agua*(((PropsSI("H", "T", T_alvo_biogas, "Q", x_vapor_agua[i+1], "Water")) - PropsSI("H", "T", T_alvo_biogas, "Q", x_vapor_agua[i], "Water")))
  L_tubo_condensacao_temp = Q_lat_vapor_agua*(((np.log(dext_tubos/dint_tubos))/(2*np.pi*k_inox))+(1/(correlacao_chen_vapor_agua(x_vapor_agua[i],'Water')*np.pi*dint_tubos))+((1/(correlacao_zukauskas_bejan(Re_casco,Pr_casco,NT,ST,SL)*np.pi*dext_tubos))))*(1/(-T_alvo_arrefecimento+T_alvo_biogas))
  L_tubo_condensacao = L_tubo_condensacao + L_tubo_condensacao_temp
  Q_cond = Q_cond + Q_lat_vapor_agua
L_tubo_condensacao
print(L_tubo_condensacao)
print(Q_cond)

1.927310597155348
24.87425081493278


In [359]:
L_tubos = L_tubos_sen + L_tubo_condensacao
L_tubos

np.float64(4.389985632856351)

In [360]:
L_serpentina

np.float64(23.103174924571643)

In [361]:
cp_aco = 486
rho_aco = 8000

In [362]:
Q_aco = (((np.pi/4)*((dext_tubos**2)-(dint_tubos**2))*L*N*rho_aco)*cp_aco*(T_ambiente-T_alvo_arrefecimento))/tempo_arrefecimento
Q_aco

np.float64(70.9463548743936)